In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from orbitfit.orbitfit import OrekitFit, STK_CONFIG
from orbitfit.utils import interp, distance_between_position, filter_with_mean_oe, plot_error_aar, get_gps_error_percentiles
from orbitfit.rotate import rotate_gps
import datetime
import copy 
import numpy as np
import logging
import json
from orbdetpy.simulation import simulate_measurements

logging.basicConfig(level=logging.INFO)

%matplotlib notebook

## Load GPS data

In [ ]:
#df_gps = pd.read_pickle('NSL1_GPS_telemetry/CH01_20210610_MTQ_Test4_rev6_rows0to1416_gps.pkl')
df_gps = pd.read_pickle('../NSL1_GPS_telemetry/CH01_20210716_GPS_Test4_Rev6-3_rows_0to686_gps.pkl')
df_gps = df_gps.tz_localize(datetime.timezone.utc)
mask = df_gps['status'] != '00'
df_gps = df_gps[mask]
df_gps = df_gps.drop(['status', 'OBT'], axis=1)

fig, ax = plt.subplots(2,1, sharex=True)
df_gps[['randv_mks_0','randv_mks_1','randv_mks_2']].plot(ax=ax[0], style='-*', grid=True)
df_gps[['randv_mks_3','randv_mks_4','randv_mks_5']].plot(ax=ax[1], style='-*', grid=True)

In [ ]:
df_gps.index[0], df_gps.index[-1]

##  Rotate and filter

In [ ]:
df_gps_eci = rotate_gps(df_gps, method="E2I")

In [ ]:
df_gps_eci_filtered = filter_with_mean_oe(df_gps_eci, thresholds={"mean_sma": 1, "mean_ecc": 0.0004, "mean_inc": 0.008}, method="median", plot=True)

## Load config file from latest optimization

In [ ]:
config_filename = "GPS_20210716/config_dict_out.json"
with open(config_filename, 'r') as fp:
    config_dict = json.load(fp)
config_dict

## Run simulation with loaded config file

In [ ]:
output_dict = simulate_measurements(config_dict)
assert "Traceback" not in output_dict, output_dict
index = pd.DatetimeIndex([o["Time"] for o in output_dict])
data = [o["TrueStateCartesian"][:6] for o in output_dict]
df_out = pd.DataFrame(data=data, index=index, columns=["randv_mks_{}".format(i) for i in range(6)])

In [ ]:
# propagated values should always be interpolated to the times of the available data.
df_out_interpolated = interp(df_out, df_gps_eci_filtered.index)
_ = plot_error_aar(df_gps_eci_filtered, df_out_interpolated)

In [ ]:
# propagated values should always be interpolated to the times of the available data.
df_out_interpolated = interp(df_out, df_gps_eci.index)
_ = plot_error_aar(df_gps_eci, df_out_interpolated)

In [ ]:
df_out_interpolated = interp(df_out, df_gps_eci.index)
get_gps_error_percentiles(df_gps_eci, df_out_interpolated, quantiles_of_interest=(0.66, 0.90, 0.99), plot=True)

## Run in loop with different values of area

In [ ]:
config_dict_aux = config_dict;
config_dict_aux["SpaceObject"]["Area"] = 0.3
output_dict = simulate_measurements(config_dict_aux)
assert "Traceback" not in output_dict, output_dict
index = pd.DatetimeIndex([o["Time"] for o in output_dict])
data = [o["TrueStateCartesian"][:6] for o in output_dict]
df_out = pd.DataFrame(data=data, index=index, columns=["randv_mks_{}".format(i) for i in range(6)])
df_out_interpolated = interp(df_out, df_gps_eci_filtered.index)
_ = plot_error_aar(df_gps_eci_filtered, df_out_interpolated)

In [ ]:
areas = np.linspace(0.08, 0.12,5)
for area in areas:
    config_dict_aux = config_dict;
    config_dict_aux["SpaceObject"]["Area"] = area
    output_dict = simulate_measurements(config_dict_aux)
    assert "Traceback" not in output_dict, output_dict
    index = pd.DatetimeIndex([o["Time"] for o in output_dict])
    data = [o["TrueStateCartesian"][:6] for o in output_dict]
    df_out = pd.DataFrame(data=data, index=index, columns=["randv_mks_{}".format(i) for i in range(6)])
    df_out_interpolated = interp(df_out, df_gps_eci_filtered.index)
    _ = plot_error_aar(df_gps_eci_filtered, df_out_interpolated)

Seems that Area = 0.1 m2 is the option that best matches the alongtrack error. But actually the best result might be 0.12 m2 because the alongtrack divergence is straight, thus meaning that the error is because of an initial position, and not because of a drag missmatch.